# Speech Recognition 기술 문서
## 음성데이터에서 텍스트로 이어지는 과정

목차
1. 요약
2. 이론적 배경
3. 패키지 설치
4. 저장된 음성 파일 사용
5. 마이크로 직접 녹음
6. 실제 비즈니스에 접목하는 방안


### 1. 요약: 
음성 파동(아날로그)을 마이크로 입력받아 디지털 신호로 변형시켜 텍스트 형식으로 출력한다.

### 2. 이론적 배경
오늘날 대부분의 음성 인식 시스템은 Hidden Markov 모델에 기반한다. 이 모델은 음성 신호를 10밀리초(ms) 구간으로 분할한다. 그리고 각 구간으로 나눠진 음성 신호를 숫자(벡터) 시퀀스로 변형시킨다. 다만 이러한 계산법은 긴 음성 신호에는 과부화될 가능성이 있다. 이를 해결하기 위해 인공신경망이 사용된다. 즉, Hidden Markov 모델을 적용하기 전에 차원 축소, feature 변형 등 일종의 전처리 과정이 실시된다. 또다른 해결방법으로 녹음기기 자체를 필요한 음성만 필터링하도록 설계할 수 있다. 

참조 문서: https://realpython.com/python-speech-recognition/

https://github.com/Uberi/speech_recognition

https://pypi.org/project/SpeechRecognition/

참조 영상: https://youtu.be/ZEQh45W_UDo?t=8985



### 3. 패키지 설치

본 문서에서는 따로 API키 발급이 필요없는 .recognize_google()을 이용한다.

In [32]:
# $ pip install speechrecognition

import speech_recognition as sr
sr.__version__

'3.8.1'

In [33]:
# Recognizer 객체 생성
r = sr.Recognizer()

## 4. 이미 저장된 녹음 파일 사용
.recognize_google() 메서드에 음성 파일을 인자로 입력하여야 한다. 호환되는 음성파일 확장자로는 .wav, .aiff, .aiff-c, .flac이 있다. 

In [34]:
file = sr.AudioFile('/Users/sollee/Desktop/havard.wav')
# 음성파일을 열어서 audio에 배정
with file as source:
    audio = r.record(source)
type(audio)

speech_recognition.AudioData

In [35]:
# 결과
r.recognize_google(audio)

"the stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle taste fine with ham tacos al Pastore are my favorite he's zestful food is be hot cross bun"

## 5. 마이크로 직접 녹음하여 사용
마이크로 음성을 인식하려면 PyAudio 패키지를 이용한다.

In [60]:
# MacOS: brew install portaudio ➜
# 공통: pip install pyaudio

mic = sr.Microphone()
sr.Microphone.list_microphone_names() # 사용 가능한 마이크 리스트 확인


['Built-in Microphone', 'Built-in Output']

In [59]:
# 마이크에 입력된 음성을 캡쳐하고 텍스트로 출력
with mic as source:
    r.adjust_for_ambient_noise(source)  # 노이즈 제거
    print('말씀해주세요: ')
    audio = r.listen(source)

    try:
        text = r.recognize_google(audio, language='kr-KO')
        print(f'>>> {text}')
    except:
        print('무슨 말씀인지 못 알아듣겠습니다')


말씀해주세요: 
>>> anyong haseyo


## 6. 한국어 인식 API
이처럼 구글 API를 사용하면 한국어를 잘 인식하지 못하고 알파벳으로 출력해주는 단점이 있다. 한국어를 정확히 인지하여 한글로 출력하려면 과학기술부에서 제공하는 오픈 API를 이용한다. 다만 API key를 사전에 받아야 하며 하루 1000건으로 제한된다. 

참조: https://aiopen.etri.re.kr/guide_recognition.php

In [ ]:
import urllib3
import json
import base64
# API key 사전 발급 필요
openApiURL = "http://aiopen.etri.re.kr:8000/WiseASR/Recognition"
accessKey = "YOUR_ACCESS_KEY"
audioFilePath = "AUDIO_FILE_PATH"
languageCode = "LANGUAGE_CODE"

# 음성 데이터를 file 변수에 저장
file = open(audioFilePath, "rb")
audioContents = base64.b64encode(file.read()).decode("utf8")
file.close()

# 음성 데이터를 JSON 포맷으로 요청
requestJson = {
    "access_key": accessKey,
    "argument": {
        "language_code": languageCode,
        "audio": audioContents
    }
}

# HTTP 통신으로 음성 파일을 API서버에 전달하고 POST방식으로 호출
http = urllib3.PoolManager()
response = http.request(
    "POST",
    openApiURL,
    headers={"Content-Type": "application/json; charset=UTF-8"},
    body=json.dumps(requestJson)
)

# 결과 출력
print("[responseCode] " + str(response.status))
print("[responBody]")
print(str(response.data,"utf-8"))

## 6. 실제 비즈니스 접목 방안
Speech-to-text 기술은 무인키오스크, 고객데이터 마이닝 등 실제 서비스화 분야가 다양하다. 

우선 카페, 음식점, 편의점 등에서 무인 키오스크의 도입이 증가하는 추세이다. 코로나19와 최저임금 상승의 여파로 당분간 이러한 추세가 이어질 것으로 보인다. 현재 대부분의 무인 키오스크는 터치스크린을 이용하고 있는데 음성 인식을 기능을 추가하여 차별화 할 수 있을 것이다. 그리고 아직 많지는 않지만 서빙 로봇에도 활용이 가능하다. 

음성인식을 적용하여 고객데이터를 모을 수도 있다. 예를들어 ARS등 고객응대 시 여러 데이터를 모아서 서비스 개선에 도움을 주는 방식이다. 즉, 음성 데이터를 토큰화 하여 어느 부분에 대한 상담이 주로 이루어지는지 분석하고 문제를 해결하는 것이다. 